In [24]:
import os
import pandas as pd
import json
import subprocess

# Question Filtering by Num Times Incorrect

In [6]:
mmlu_counts = {'ethics' : {}, 'safety' : {}, 'metacognition' : {}}

for f in os.listdir('../../out/gpt4o/mas/'):
    if 'RESPONSE' in f and 'mmlu_' in f:
        df_responses = pd.read_csv('../../out/gpt4o/mas/' + f).set_index('id')
        df_category = f.split('_')[4]
        for index, row in df_responses.iterrows():
            if row['is_correct'] == 0:
                mmlu_counts[df_category][index] = mmlu_counts[df_category].get(index, 0) + 1

for f in os.listdir('../../out/gpt4o/evaloptim/'):
    if 'RESPONSE' in f and 'mmlu_' in f:
        df_responses = pd.read_csv('../../out/gpt4o/evaloptim/' + f).set_index('id')
        df_category = f.split('_')[1]
        for index, row in df_responses.iterrows():
            if row['is_correct'] == 0:
                mmlu_counts[df_category][index] = mmlu_counts[df_category].get(index, 0) + 1

for f in os.listdir('../../out/gpt4o/zeroshot/'):
    if 'RESPONSE' in f and 'mmlu_' in f:
        df_responses = pd.read_csv('../../out/gpt4o/zeroshot/' + f).set_index('id')
        df_category = f.split('_')[2]
        for index, row in df_responses.iterrows():
            if row['is_correct'] == 0:
                mmlu_counts[df_category][index] = mmlu_counts[df_category].get(index, 0) + 1




In [7]:
threshold = 3
thresh_ids = {'ethics': [], 'safety': [], 'metacognition': []}
for category, counts in mmlu_counts.items():
    print(f"Category: {category}")
    for index, count in counts.items():
        if count >= threshold:
            thresh_ids[category].append(index)
    print(f"{len(thresh_ids[category])} IDs with at least {threshold}")
    thresh_ids[category].sort()
    print()  # Add a newline for better readability between categories

Category: ethics
0 IDs with at least 3

Category: safety
0 IDs with at least 3

Category: metacognition
0 IDs with at least 3



In [ ]:
#json.dump(thresh_ids, open('../../out/gpt4o/mmlu_incorrect_threshold_3.json', 'w'))

In [9]:
thresh_ids = json.load(open('../../out/gpt4o/mmlu_incorrect_threshold_3.json', 'r'))

# Log Parsing for "Difficult Questions" 

In [22]:
log_paths = {'mas': {'ethics': '/mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs', 
                     'safety': '/mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_safety_gpt4o_mmlu_safety/logs', 
                     'metacognition': '/mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_metacognition_gpt4o_mmlu_metacognition/logs'},
              'evaloptim': {'ethics': '/mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/evaloptimizer_mmlu_ethics/logs',
                            'safety': '/mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/evaloptimizer_gpt4o_mmlu_safety/logs',
                            'metacognition': '/mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/evaloptimizer_gpt4o_mmlu_metacognition/logs'}
        }

In [27]:
for category in log_paths:
    for subset in log_paths[category]:
        print(category.upper(), subset.upper())
        for logfile in os.listdir(log_paths[category][subset]):
            if logfile.endswith('.txt') and int(logfile.split('.txt')[0]) in thresh_ids[subset]:
                print(logfile, category, log_paths[category][subset])
                from_path = os.path.join(log_paths[category][subset], logfile)
                to_path = os.path.join('/mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/gpt4o/mmlu_thresh_analysis', category, subset, logfile)
                subprocess.run(['cp', from_path, to_path])
        print()

MAS ETHICS
25.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs
612.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs
118.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs
854.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs
885.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs
350.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs
22.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs
163.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethics/logs
502.txt mas /mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/mas_ethics_gpt4o_mmlu_ethi

In [26]:
!pwd

/mnt/bulk-sirius/sanddhya/agent_benchmarking/Agent_Edge_Benchmarking/code/analysis
